In [1]:
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import json
import os
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [179]:
from bs4 import BeautifulSoup as bs
import requests
from langdetect import detect

# Création de la liste URL

In [136]:
def listeUR () :
    racineURL = "https://www.youtube.com/results?search_query="
    EquationRecherche = "airbus+A380"

    r = requests.get(racineURL + EquationRecherche)
    page = r.text
    soup = bs(page,'html.parser')

    videos = soup.findAll('a',attrs={'class':'yt-uix-tile-link'})
    listVideos=[]
    for v in videos:
        URL = 'https://www.youtube.com' + v['href']
        listVideos.append(URL)
    return listVideos

# Fonction qui nous crée notre fichier Json remplie

In [232]:
def CréeJs(listeURL,chiffre,commentaire,nb_com) :
    nom_col = ['Airline_Type','Region_Operation','Aircraft_Type','Cabin_Class','Type_Of_Lounge',
               'Type_Of_Traveller','Date_Visit','Date_Flown','Airport','Route','Category','Category_Detail',
               'Cabin_Staff_Service','Lounge_Staff_Service','Bar_And_Beverages','Food_And_Beverages','Ground_Service','Catering','Cleanliness',
              'Lounge_Comfort','Aisle_Space','Wifi_And_Connectivity','Inflight_Entertainment','Viewing_Tv_Screen','Power_Supply',
              'Seat','Seat_type','Seat_Comfort','Seat_Legroom','Seat_Storage','Seat_Width','Seat_Recline','Washrooms',
               'Value_For_Money','Overall_Customer_Rating','Overall_Service_Rating','Overall_Airline_Rating',
              'Recommended','Departure_city','Arrival_city','Nb_bus_taken','Nb_train_taken',
               'Nb_car_taken','Nb_plane_taken','Duration','Price_min','Price_max','Nb_sharing','Awards','Registration','Language']
    
    url = listeURL[chiffre]
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    video_details = {}
    
#Fill data

    video_details['Data_Source'] = 'Youtube'
    
    for i in range (38) : 
        video_details[nom_col[i]] = ' '
    
    video_details['Date_Review'] = commentaire
    video_details['Review'] = commentaire
    
    for i in range (38,47) :
        video_details[nom_col[i]] = ' '
        
#sortir le titre de la vidéo
    video_details['Title'] = soup.find('span',attrs={'class': 'watch-title'}).text.strip()

#sortir le nom de la chaine
    for script in soup.findAll('script',attrs={'type': 'application/ld+json'}):
        channelDescription = json.loads(script.text.strip())
        video_details['Author'] = channelDescription['itemListElement'][0]['item']['name']
        
#sortir description
    video_details['Description'] = soup.find('p',attrs={'id': "eow-description"}).text.strip()
    
#sortir la date de publication 
    video_details['Date_publication'] = soup.find('strong',attrs={'class': "watch-time-text"}).text.strip()
    
#sortir le nombre de vue
    video_details['View_Count'] =soup.find('div',attrs={'class': 'watch-view-count'}).text.strip()

#sortir un bouton likes
    for span in soup.findAll('',attrs={'class':"yt-uix-button yt-uix-button-size-default yt-uix-button-opacity yt-uix-button-has-icon no-icon-markup like-button-renderer-like-button like-button-renderer-like-button-unclicked yt-uix-clickcard-target yt-uix-tooltip"}):
        video_details['Likes']=span.find('span',attrs={'class':'yt-uix-button-content'}).text.strip()


#sortir un bouton de dislikes 
    for button in soup.findAll('button',attrs={'class':"yt-uix-button yt-uix-button-size-default yt-uix-button-opacity yt-uix-button-has-icon no-icon-markup like-button-renderer-dislike-button like-button-renderer-dislike-button-unclicked yt-uix-clickcard-target yt-uix-tooltip"}):
        video_details['Dislikes']=button.find('span',attrs={'class':'yt-uix-button-content'}).text.strip()

#sortir le nombre d'abonné
    if (soup.find('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}) == None):
        video_details["Nb_subscribers"] = 0
    else :
        video_details["Nb_subscribers"] = soup.find('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}).text.strip()
        

    video_details['Nb_comments'] = nb_com

    
    for i in range (47,50):
        video_details[nom_col[i]] = ' '
    
    #detect(str(commentaire[0]))
    video_details['Language'] = 'fr'
    

#sortir hashtags
    hashtags = []

    for span in soup.findAll('span',attrs={'class': 'standalone-collection-badge-renderer-text'}):
        for a in span.findAll('a',attrs={'class': 'yt-uix-sessionlink'}):
            hashtags.append(a.text.strip())
    video_details['hashtags'] = hashtags
    with open('data.json', 'a', encoding='utf8') as outfile:
        json.dump(video_details, outfile, ensure_ascii=False,indent=4)

        
        
def scroll (url,nb_scroll):
    options = Options() 
    options.add_argument("--start-maximized")
    options.add_argument('headless')
    driver = webdriver.Chrome("/home/sid2019-13/Téléchargements/chromedriver",chrome_options=options)
    driver.get(url)

    Y=0
    for _ in range(nb_scroll):
        time.sleep(2)
        driver.execute_script("window.scrollTo("+str(Y)+","+str(Y+800)+")")
        Y+=800
    return BeautifulSoup(driver.page_source,'html.parser')

# Implémentation

In [234]:
for v in range(len(listVideos)):
    soup = scroll(listVideos[v],4)
    for span in soup.findAll('yt-formatted-string',attrs={'class': 'style-scope ytd-comment-renderer'}):
        if span.text.strip() != '' and span.text.strip() != None:
            commentaire = span.text.strip()
            nb_com = soup.find('yt-formatted-string',attrs={'class': 'count-text style-scope ytd-comments-header-renderer'}).text.strip()
            CréeJs(listVideos,v,commentaire,nb_com)
    
print('fini')

/home/sid2019-13/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options


AttributeError: 'NoneType' object has no attribute 'text'

# poubelle

In [55]:
print(soup)

<html lang="fr-FR" style="font-size: 10px;font-family: Roboto, Arial, sans-serif; "><head><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflM4IzVZfr_FRannotations_modulejs" data-original-src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/annotations_module.js" src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/annotations_module.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflM4IzVZfr_FRendscreenjs" data-original-src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/endscreen.js" src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/endscreen.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflM4IzVZfr_FRcaptionsjs" data-original-src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/captions.js" src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/captions.js"></script><script class="js-httpswwwyoutubecomytsjsbinplayer_iasvflM4IzVZfr_FRminiplayerjs" data-original-src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/miniplayer.js" src="/yts/jsbin/player_ias-vflM4IzVZ/fr_FR/miniplayer.js"></script><script cla

In [212]:
soup = scroll(listVideos[0],4)
for span in soup.findAll('yt-formatted-string',attrs={'class': 'style-scope ytd-comment-renderer'}):
    if span.text.strip() != '' or span.text.strip() != None:
        commentaire = span.text.strip()
        CréeJs(listVideos,v,commentaire)

/home/sid2019-13/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:89: DeprecationWarning: use options instead of chrome_options


AttributeError: 'NoneType' object has no attribute 'text'

In [223]:

for span in soup.findAll('yt-formatted-string',attrs={'class': 'style-scope ytd-comment-renderer'}):
    if span.text.strip() != '' and span.text.strip() != None:
        commentaire = span.text.strip()
        print(commentaire)
        CréeJs(listVideos,v,commentaire)
    

7:02 les volets c'est pour que l'avion change de direction ? 😅


AttributeError: 'NoneType' object has no attribute 'text'

In [50]:

url = 'https://www.youtube.com/watch?v=N8mTPYiCO-8'
import lxml.html

mysite = urllib.request.urlopen(url).read()
lxml_mysite = lxml.html.fromstring(mysite)

description = lxml_mysite.xpath("//meta[@name='description']")[0] # meta tag description
text = description.get('content') # content attribute of the tag

print(mysite)

b'<!DOCTYPE html><html lang="fr-FR" data-cast-api-enabled="true"><head><style name="www-roboto" >@font-face{font-family:\'Roboto\';font-style:normal;font-weight:400;src:local(\'Roboto Regular\'),local(\'Roboto-Regular\'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:normal;font-weight:500;src:local(\'Roboto Medium\'),local(\'Roboto-Medium\'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:italic;font-weight:400;src:local(\'Roboto Italic\'),local(\'Roboto-Italic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:italic;font-weight:500;src:local(\'Roboto Medium Italic\'),local(\'Roboto-MediumItalic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOjCnqEu92Fr1Mu51S7ACc6CsE.ttf)format(\'truetype\');}</style><script name="www-roboto" >if (document.fo

In [28]:
https://developers.google.com/youtube/v3/docs/playlists/list#try-it_1

SyntaxError: invalid syntax (<ipython-input-28-ce79a0560219>, line 1)

In [27]:
import time
#import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv

browser = webdriver.Chrome()

#change your channel url here

browser.get("https://www.youtube.com/user/googleindia/videos")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")


#considering the number of videos in your channel change this below figure

no_of_loadmore = 100

try:
    while no_of_loadmore:
        browser.find_element_by_class_name("load-more-text").click()
        time.sleep(1)
        no_of_loadmore-=1
except:
    pass

#scrape the content by xpath

post_elems = browser.find_elements_by_xpath('//div/h3/a')
links = browser.find_elements_by_xpath('//*[@id="channels-browse-content-grid"]/li/div/div[1]/div[2]/h3/a')

with open('googleindia_vids.csv', 'w',newline='') as csvfile:
    autowriter = csv.writer(csvfile)
    autowriter.writerow(['Title','URL'])
    for i in range(0,len(post_elems)-1):
        autowriter.writerow([post_elems[i].text,links[i].get_attribute('href')])

#iterate thru the elements and print it

for post in post_elems:
  print(post.text) 

for link in links:
    print(link.get_attribute('href'))

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [26]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

BASE_URL = "https://www.youtube.com"
BASE_SEARCH_URL = "https://www.youtube.com/results?search_query="

query = "airbus+A380"

r = requests.get(BASE_SEARCH_URL + query)
page = r.text
soup = bs(page, 'html.parser').find_element_by_tag_name('html').send_keys(Keys.END)

vids = soup.findAll('a',attrs={'class':'yt-uix-tile-link'})
video_urls = [BASE_URL + vid['href'] for vid in vids]



print(len(video_urls))
print(video_urls[29])






TypeError: 'NoneType' object is not callable

In [9]:
import requests

 
for script in soup.findAll('script'):
    requests.get(script.text.strip()).json()

MissingSchema: Invalid URL 'if (document.fonts && document.fonts.load) {document.fonts.load("400 10pt Roboto", "F");document.fonts.load("500 10pt Roboto", "F");}': No schema supplied. Perhaps you meant http://if (document.fonts && document.fonts.load) {document.fonts.load("400 10pt Roboto", "F");document.fonts.load("500 10pt Roboto", "F");}?

In [10]:

for script in soup.findAll('script'):
        test = json.loads(script.text.strip())
        print(test)
# FAUX  
com = []
for span in soup.findAll('a',attrs={'class': 'yt-formatted-string'}):
    print(span)
    for a in span.findAll('span',attrs={'class': 'style-scope ytd-comment-renderer'}):
        print('ok')
        com.append(a.text.strip())
video_details['Commentaires'] = com
for link in soup.findAll(id="content-text"):
    print(link.get_text())



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [12]:
r = requests.get("https://www.youtube.com/watch?v=2RDCppcfveg")
page = r.text
page

'<!DOCTYPE html><html lang="fr-FR" data-cast-api-enabled="true"><head><style name="www-roboto" >@font-face{font-family:\'Roboto\';font-style:normal;font-weight:500;src:local(\'Roboto Medium\'),local(\'Roboto-Medium\'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:normal;font-weight:400;src:local(\'Roboto Regular\'),local(\'Roboto-Regular\'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:italic;font-weight:500;src:local(\'Roboto Medium Italic\'),local(\'Roboto-MediumItalic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOjCnqEu92Fr1Mu51S7ACc6CsE.ttf)format(\'truetype\');}@font-face{font-family:\'Roboto\';font-style:italic;font-weight:400;src:local(\'Roboto Italic\'),local(\'Roboto-Italic\'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format(\'truetype\');}</style><script name="www-roboto" >if (document.fon

In [ ]:
from selenium import webdriver
import time
import re
import csv
import string

def isEnglish(s):
    try:
        s.encode('ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

driver = webdriver.Chrome('C:\webdrivers\chromedriver')
yt_link = input("Link to Youtube video: ")
print("-------------------------------------------------------------------------------------------------------------------")
driver.get(yt_link)
driver.maximize_window()
time.sleep(5)
title = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
print("Video Title: " + title)
print("-------------------------------------------------------------------------------------------------------------------")

comment_section = driver.find_element_by_xpath('//*[@id="comments"]')
driver.execute_script("arguments[0].scrollIntoView();", comment_section)
time.sleep(7)

last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait to load page
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)

name_elems=driver.find_elements_by_xpath('//*[@id="author-text"]')
comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
num_of_names = len(name_elems)
for i in range(num_of_names):
    username = name_elems[i].text    # .replace(",", "|")
    # username = emoji_pattern.sub(r'', username)
    # username = str(username).replace("\n", "---")
    comment = comment_elems[i].text    # .replace(",", "|")
    # comment = emoji_pattern.sub(r'', comment)
    # comment = str(comment).replace("\n", "---")
    
    #if isEnglish(comment) == False:
     #   comment = "NOT ENGLISH"
        
    print(username + ": " + comment) # comment.translate({ord(i):None for i in '' if i not in string.printable})
    print("-------------------------------------------------------------------------------------------------------------------")

driver.close()